# InSAR Processing Using ASF HYP3 on-demand products

## Getting Started

The Alaska Satellite Facility (ASF) archives SAR and InSAR data on behalf of NASA. 
By searching on their [Vertex website](https://search.asf.alaska.edu/#/?maxResults=250), users can access and download the original data, or in some cases can run basic processing. 
Vertex also allows users to do basic SBAS processing, which produces interferograms that can be downloaded and processed to time-series. 
This is what we will in this exercise. 


The ASF User guide can be found [here](https://docs.asf.alaska.edu/vertex/manual/)

See the HYP3 SDK [here](https://hyp3-docs.asf.alaska.edu/using/sdk/)

In [1]:
%pip install hyp3_sdk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import netrc
from pathlib import Path

import numpy as np
import rasterio as rio
import resample_data
from hyp3_sdk import HyP3


In [ ]:
creds = netrc.netrc().authenticators('urs.earthdata.nasa.gov')
assert creds is not None
username, _, password = creds

hyp3 = HyP3(username=username, password=password)
del password
batch = hyp3.find_jobs()

Once you have complete jobs you can download the products to your machine

In [25]:
batch.complete()

True

### WARNING!!!
This step will download files from ASF. 
These files vary in size but are roughly ~0.5 GB each, so 200 will be ~100 GB. 
Make sure you have adequete space for downloading before starting! 
The good news is that file downloads don't repeat, so if the code stops for any reason you can restart from where it ended. 

In [9]:
# specify your download folder
dload_path_name = Path('asf_insar_data')

In [10]:
batch.download_files(dload_path_name, create=True)

0it [00:00, ?it/s]


[]

### Steps
The first steps are to download and unzip the products in our working directory. In this case, I have already done this step. 

In [11]:
%cd {dload_path_name}

[Errno 2] No such file or directory: 'asf_insar_data'
/mnt/g/Garlic/Documents/Code/GitHub/gemlab/software/insar


In [12]:
%ls

Cf_InSAR_and_GNSS_velocities.ipynb*      makeTS.py*
convert_h5_to_kml.py*                    process_data.py*
InSAR_start.ipynb*                       __pycache__/
InSAR_with_ASF_ondemand_products.ipynb*  zipped/


In [13]:
# Unzip your folders
# Depending on your platform this may not work
!unzip '*.zip'

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [14]:
%ls

Cf_InSAR_and_GNSS_velocities.ipynb*      makeTS.py*
convert_h5_to_kml.py*                    process_data.py*
InSAR_start.ipynb*                       __pycache__/
InSAR_with_ASF_ondemand_products.ipynb*  zipped/


As you can see, we have a number of folders that each contain a set of files, including unwrapped phase, unwrapped displacements, DEM, look vectors, etc. 

Each folder is named by the two dates that were used to generate that interferogram, so for example `S1AA_20220420T235621_20220514T235622_VVP024_INT80_G_ueF_CF44` is an interferogram for dates 4/20/2022 and 5/14/2024. 

In [15]:
# Count the number of interferograms we have downloaded
# If this line does not work on your machine feel free to skip
%ls -ld */ | wc -l  # this will count folders only

2


In [16]:
%mkdir zipped

mkdir: cannot create directory ‘zipped’: File exists


In [17]:
%mv *.zip zipped/.

mv: cannot stat '*.zip': No such file or directory


In [18]:
%ls

Cf_InSAR_and_GNSS_velocities.ipynb*      makeTS.py*
convert_h5_to_kml.py*                    process_data.py*
InSAR_start.ipynb*                       __pycache__/
InSAR_with_ASF_ondemand_products.ipynb*  zipped/


In [19]:
# some QA/QC
%ls S1*/*_unw_phase.tif | wc -l

ls: cannot access 'S1*/*_unw_phase.tif': No such file or directory
0


In [20]:
# this should match the previous
%ls S1*/*_corr.tif | wc -l 

ls: cannot access 'S1*/*_corr.tif': No such file or directory
0


In [21]:
# check file sizes
unw_paths = list(dload_path_name.glob('S1*/*_unw_phase.tif'))
corr_paths = list(dload_path_name.glob('S1*/*corr.tif'))
print(len(unw_paths))
print(len(corr_paths))

all_heights: list[float] = []
all_widths: list[float] = []
for path in unw_paths:
    with rio.open(path) as f:
        all_heights.append(f.height)
        all_widths.append(f.width)

print(np.mean(all_heights))
print(np.mean(all_widths))

0
0
nan
nan


/home/garlic/.conda/envs/RAiDER-13/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/garlic/.conda/envs/RAiDER-13/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [22]:
assert len(set(unw_paths)) == len(unw_paths)

The next step is to make sure that all the interferograms have the same shape and geographic extent. 
We will ensure this using a simple code that uses rasterio to re-project all the interferograms to match a reference. 
For this code, you will need `rasterio` and `numpy` installed. 

In [23]:
# for k, f in enumerate(files):
#     with rio.open(f) as F:
#         plt.plot([k,k], F.

In [2]:
# The function to resize the images is called run_resampling
shp_file_path = Path('~/CEF/Projects/SLU/t92f120/asf_insar_data/stl_polygon/stl_polygon.shp')
assert shp_file_path.exists()
resample_data.main(shapefile_path=shp_file_path)  # runs on the current directory by default

AssertionError: 

Once the resampling has finished, we can now proceed with time-series analysis as normal!
You can refer to the [InSAR lab](https://github.com/jlmaurer/GE6146/blob/master/notebooks/Lab4_InSAR.ipynb) for more details and plotting examples. 

In [ ]:
# We'll use the same functions as for the lab
import makeTS

In [ ]:
dir = Path.cwd()
ifg_paths = list(dir.glob('**/*_unw_phase.tif'))
coh_paths = list(dir.glob('**/*_corr.tif'))

In [ ]:
print(f'I found {len(ifg_paths)} interferograms')

I found 771 interferograms


In [ ]:
# Pull all the data
date_pairs, dates = makeTS.get_dates(ifg_paths)
year_fracs = np.array([makeTS.get_year_frac(d) for d in dates])
g_matrix = makeTS.make_g_matrix(dates, date_pairs)
x_size, y_size, data_type, geo_proj, transform, nodata_val, num_bands = makeTS.read_raster(ifg_paths[0])
data = makeTS.get_data(ifg_paths, 1)

In [ ]:
print(f'Interferograms are {y_size}x{x_size} pixels in size')

Interferograms are 2841x3557 pixels in size


In [ ]:
# You'll need to make a plot of one of the dates or the average coherence so you can pick a reference point

In [ ]:
# De-referencing
data = makeTS.dereference(data, ref_center=[2500, 2000], ref_size=10)

In [ ]:
# Create the time-series data cube
ts_array = makeTS.make_ts(g_matrix, data)

In [ ]:
# Convert to displacement in meters
ts_array = makeTS.radians_to_meters(ts_array, lam=0.056)  # wavelength lam is 5.6 cm for Sentinel-1

In [ ]:
# Compute the mean velocity by fitting a line
vel = makeTS.find_mean_vel(ts_array, year_fracs)

In [ ]:
vel.shape

(2841, 3557)

### Saving Data
Want to change this to use NETCDF and xarray, include time-series and coherence time-series in the files, maybe IFGs and coherence ifgs as well. 

In [ ]:
import h5py


with h5py.File('velocity.h5') as path:
    for key in path.keys():
        print(key)

intercept
interceptStd
residue
velocity
velocityStd


## Tropospheric correction using RAiDER

TBD